<a href="https://colab.research.google.com/github/sowmyarshetty/Project2-AI-Class/blob/main/Project2_AI_Class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# Import the data
import pandas as pd
import numpy as np
from sklearn.preprocessing import  OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import gdown
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
#URL for downloading lung cancer data
url_lc = 'https://drive.google.com/uc?id=1qQSYbhrUczPKIj22h0AC2f9rT4ljI3BC'
gdown.download(url_lc, 'lung_cancer_prediction_dataset.csv', quiet=False)


#Read the CarRentalData.csv
df_lc = pd.read_csv(url_lc)
df_lc.head()


Downloading...
From: https://drive.google.com/uc?id=1qQSYbhrUczPKIj22h0AC2f9rT4ljI3BC
To: /content/lung_cancer_prediction_dataset.csv
100%|██████████| 23.5M/23.5M [00:00<00:00, 25.3MB/s]


,ID,Country,Population_Size,Age,Gender,Smoker,Years_of_Smoking,Cigarettes_per_Day,Passive_Smoker,Family_History,...,Air_Pollution_Exposure,Occupational_Exposure,Indoor_Pollution,Healthcare_Access,Early_Detection,Treatment_Type,Developed_or_Developing,Annual_Lung_Cancer_Deaths,Lung_Cancer_Prevalence_Rate,Mortality_Rate
0,0,China,1400,80,Male,Yes,30,29,No,No,...,Low,Yes,No,Poor,No,NaN,Developing,690000,2.44,0.0
1,1,Iran,84,53,Male,No,0,0,Yes,No,...,Low,Yes,No,Poor,No,NaN,Developing,27000,2.10,0.0
2,2,Mexico,128,47,Male,Yes,12,6,Yes,No,...,Medium,No,No,Poor,Yes,NaN,Developing,28000,1.11,0.0
3,3,Indonesia,273,39,Female,No,0,0,No,Yes,...,Low,No,No,Poor,No,NaN,Developing,40000,0.75,0.0
4,4,South Africa,59,44,Female,No,0,0,Yes,No,...,Medium,Yes,No,Poor,No,NaN,Developing,15000,2.44,0.0


In [17]:
df_lc.dtypes

,0
ID,int64
Country,object
Population_Size,int64
Age,int64
Gender,object
Smoker,object
Years_of_Smoking,int64
Cigarettes_per_Day,int64
Passive_Smoker,object
Family_History,object


#Data Pre-processing
*   Identify the columns in the data set  
## Scikit-learn encoding methods
*   unsupervised learning
*   
*   






# Train Test Split

*   After encoding categorical data , work on the encoded car_rental data
*   Identify the X columns ( i.e features or the independent variables)
*   Identify the Y column ( i.e in our case price)
*   Use the Sklearn `train_test_split()` function to split the data into training and testing data


*   Create the LinearRegression Model
*   Evaluate the model
*   Use our model to make predictions
*   Score the predictions with mse and r2
*   Call the score() method on the model to show the r2 score














# Compare performance with linear regression model
*   Lasso Regression
*   Ridge regression

